In [ ]:
%pylab inline

%load_ext autoreload
%autoreload 2

import pandas as pd
from py2neo import Graph

In [ ]:
graph = Graph('http://localhost:7474')

In [ ]:
query = 'MATCH p = (source)--(destination) RETURN source, source.name, source.known_terrorist, avg(destination.known_terrorist)'
df = pd.DataFrame(graph.data(query))
df.head()

query permutations
- directedness
- levels of depth
- type of connection

In [ ]:
import itertools

In [ ]:
list(itertools.product(*[['text', 'call', 'both'],
                         ['1','2','3', '1..2', '1..3'],
                         ['undirected', 'src->dst', 'src<-dst']]))